Script to analyse data related to fisheries. These data concerns:

Target 14.4 (FMSY/F and B/BMSY)

Target 14.6 (TAC/Catch)

Target 14.a (SAD/TAC) 

In [119]:
import os
import pandas as pd
import numpy as np

In [33]:
pd.set_option('display.max_columns', 500)

In [41]:
os.chdir('..')
os.getcwd()

'c:\\Users\\isaldiviagonzatti\\OneDrive - Kiel Institut für Weltwirtschaft\\Documents\\General\\Ignacio'

In [42]:
countries=['Belgium','Bulgaria','Cyprus', 'Greece','Germany','Croatia','Italy', 
           'Denmark','Estonia','Spain','Finland','France','Ireland','Lithuania',
           'Latvia','Malta','Netherlands','Poland','Portugal', 'Romania',
           'Sweden','United Kingdom of Great Britain and Northern Ireland', "United Kingdom of GB"]

In [43]:
country_to_abbrev = {
    "Andorra": "AD",
    "United Arab Emirates": "AE",
    "Afghanistan": "AF",
    "Antigua and Barbuda": "AG",
    "Anguilla": "AI",
    "Albania": "AL",
    "Armenia": "AM",
    "Angola": "AO",
    "Antarctica": "AQ",
    "Argentina": "AR",
    "American Samoa": "AS",
    "Austria": "AT",
    "Australia": "AU",
    "Aruba": "AW",
    "Åland Islands": "AX",
    "Azerbaijan": "AZ",
    "Bosnia and Herzegovina": "BA",
    "Barbados": "BB",
    "Bangladesh": "BD",
    "Belgium": "BE",
    "Burkina Faso": "BF",
    "Bulgaria": "BG",
    "Bahrain": "BH",
    "Burundi": "BI",
    "Benin": "BJ",
    "Saint Barthélemy": "BL",
    "Bermuda": "BM",
    "Brunei Darussalam": "BN",
    "Bolivia (Plurinational State of)": "BO",
    "Bonaire, Sint Eustatius and Saba": "BQ",
    "Brazil": "BR",
    "Bahamas": "BS",
    "Bhutan": "BT",
    "Bouvet Island": "BV",
    "Botswana": "BW",
    "Belarus": "BY",
    "Belize": "BZ",
    "Canada": "CA",
    "Cocos (Keeling) Islands": "CC",
    "Congo, Democratic Republic of the": "CD",
    "Central African Republic": "CF",
    "Congo": "CG",
    "Switzerland": "CH",
    "Côte d'Ivoire": "CI",
    "Cook Islands": "CK",
    "Chile": "CL",
    "Cameroon": "CM",
    "China": "CN",
    "Colombia": "CO",
    "Costa Rica": "CR",
    "Cuba": "CU",
    "Cabo Verde": "CV",
    "Curaçao": "CW",
    "Christmas Island": "CX",
    "Cyprus": "CY",
    "Czechia": "CZ",
    "Germany": "DE",
    "Djibouti": "DJ",
    "Denmark": "DK",
    "Dominica": "DM",
    "Dominican Republic": "DO",
    "Algeria": "DZ",
    "Ecuador": "EC",
    "Estonia": "EE",
    "Egypt": "EG",
    "Western Sahara": "EH",
    "Eritrea": "ER",
    "Spain": "ES",
    "Ethiopia": "ET",
    "Finland": "FI",
    "Fiji": "FJ",
    "Falkland Islands (Malvinas)": "FK",
    "Micronesia (Federated States of)": "FM",
    "Faroe Islands": "FO",
    "France": "FR",
    "Gabon": "GA",
    "United Kingdom of Great Britain and Northern Ireland": "UK", #original is GB, Eurostat uses UK
    "United Kingdom of GB": "GB",
    "Grenada": "GD",
    "Georgia": "GE",
    "French Guiana": "GF",
    "Guernsey": "GG",
    "Ghana": "GH",
    "Gibraltar": "GI",
    "Greenland": "GL",
    "Gambia": "GM",
    "Guinea": "GN",
    "Guadeloupe": "GP",
    "Equatorial Guinea": "GQ",
    "Greece": "EL", #original ir GR, Eurostat uses EL
    "South Georgia and the South Sandwich Islands": "GS",
    "Guatemala": "GT",
    "Guam": "GU",
    "Guinea-Bissau": "GW",
    "Guyana": "GY",
    "Hong Kong": "HK",
    "Heard Island and McDonald Islands": "HM",
    "Honduras": "HN",
    "Croatia": "HR",
    "Haiti": "HT",
    "Hungary": "HU",
    "Indonesia": "ID",
    "Ireland": "IE",
    "Israel": "IL",
    "Isle of Man": "IM",
    "India": "IN",
    "British Indian Ocean Territory": "IO",
    "Iraq": "IQ",
    "Iran (Islamic Republic of)": "IR",
    "Iceland": "IS",
    "Italy": "IT",
    "Jersey": "JE",
    "Jamaica": "JM",
    "Jordan": "JO",
    "Japan": "JP",
    "Kenya": "KE",
    "Kyrgyzstan": "KG",
    "Cambodia": "KH",
    "Kiribati": "KI",
    "Comoros": "KM",
    "Saint Kitts and Nevis": "KN",
    "Korea (Democratic People's Republic of)": "KP",
    "Korea, Republic of": "KR",
    "Kuwait": "KW",
    "Cayman Islands": "KY",
    "Kazakhstan": "KZ",
    "Lao People's Democratic Republic": "LA",
    "Lebanon": "LB",
    "Saint Lucia": "LC",
    "Liechtenstein": "LI",
    "Sri Lanka": "LK",
    "Liberia": "LR",
    "Lesotho": "LS",
    "Lithuania": "LT",
    "Luxembourg": "LU",
    "Latvia": "LV",
    "Libya": "LY",
    "Morocco": "MA",
    "Monaco": "MC",
    "Moldova, Republic of": "MD",
    "Montenegro": "ME",
    "Saint Martin (French part)": "MF",
    "Madagascar": "MG",
    "Marshall Islands": "MH",
    "North Macedonia": "MK",
    "Mali": "ML",
    "Myanmar": "MM",
    "Mongolia": "MN",
    "Macao": "MO",
    "Northern Mariana Islands": "MP",
    "Martinique": "MQ",
    "Mauritania": "MR",
    "Montserrat": "MS",
    "Malta": "MT",
    "Mauritius": "MU",
    "Maldives": "MV",
    "Malawi": "MW",
    "Mexico": "MX",
    "Malaysia": "MY",
    "Mozambique": "MZ",
    "Namibia": "NA",
    "New Caledonia": "NC",
    "Niger": "NE",
    "Norfolk Island": "NF",
    "Nigeria": "NG",
    "Nicaragua": "NI",
    "Netherlands": "NL",
    "Norway": "NO",
    "Nepal": "NP",
    "Nauru": "NR",
    "Niue": "NU",
    "New Zealand": "NZ",
    "Oman": "OM",
    "Panama": "PA",
    "Peru": "PE",
    "French Polynesia": "PF",
    "Papua New Guinea": "PG",
    "Philippines": "PH",
    "Pakistan": "PK",
    "Poland": "PL",
    "Saint Pierre and Miquelon": "PM",
    "Pitcairn": "PN",
    "Puerto Rico": "PR",
    "Palestine, State of": "PS",
    "Portugal": "PT",
    "Palau": "PW",
    "Paraguay": "PY",
    "Qatar": "QA",
    "Réunion": "RE",
    "Romania": "RO",
    "Serbia": "RS",
    "Russian Federation": "RU",
    "Rwanda": "RW",
    "Saudi Arabia": "SA",
    "Solomon Islands": "SB",
    "Seychelles": "SC",
    "Sudan": "SD",
    "Sweden": "SE",
    "Singapore": "SG",
    "Saint Helena, Ascension and Tristan da Cunha": "SH",
    "Slovenia": "SI",
    "Svalbard and Jan Mayen": "SJ",
    "Slovakia": "SK",
    "Sierra Leone": "SL",
    "San Marino": "SM",
    "Senegal": "SN",
    "Somalia": "SO",
    "Suriname": "SR",
    "South Sudan": "SS",
    "Sao Tome and Principe": "ST",
    "El Salvador": "SV",
    "Sint Maarten (Dutch part)": "SX",
    "Syrian Arab Republic": "SY",
    "Eswatini": "SZ",
    "Turks and Caicos Islands": "TC",
    "Chad": "TD",
    "French Southern Territories": "TF",
    "Togo": "TG",
    "Thailand": "TH",
    "Tajikistan": "TJ",
    "Tokelau": "TK",
    "Timor-Leste": "TL",
    "Turkmenistan": "TM",
    "Tunisia": "TN",
    "Tonga": "TO",
    "Turkey": "TR",
    "Trinidad and Tobago": "TT",
    "Tuvalu": "TV",
    "Taiwan, Province of China": "TW",
    "Tanzania, United Republic of": "TZ",
    "Ukraine": "UA",
    "Uganda": "UG",
    "United States Minor Outlying Islands": "UM",
    "United States of America": "US",
    "Uruguay": "UY",
    "Uzbekistan": "UZ",
    "Holy See": "VA",
    "Saint Vincent and the Grenadines": "VC",
    "Venezuela (Bolivarian Republic of)": "VE",
    "Virgin Islands (British)": "VG",
    "Virgin Islands (U.S.)": "VI",
    "Viet Nam": "VN",
    "Vanuatu": "VU",
    "Wallis and Futuna": "WF",
    "Samoa": "WS",
    "Yemen": "YE",
    "Mayotte": "YT",
    "South Africa": "ZA",
    "Zambia": "ZM",
    "Zimbabwe": "ZW",
}
    
# invert the dictionary
abbrev_to_country = dict(map(reversed, country_to_abbrev.items()))

### icesTACcomparison

In [44]:
pd.ExcelFile(os.path.realpath("data/icesTACcomparison.xlsx")).sheet_names

['Menus',
 'Table of contents',
 'ICES advice',
 'Council agreed TAC',
 'Comparison',
 'Table for results',
 'Overall results',
 'Results by Member State',
 'Sea basin',
 'Third country',
 'Results by % difference',
 'Results by # of TACs',
 'Results by # of TACs by MS',
 'Results by species',
 'ID',
 'Matching - ICES-TAC',
 'Matching - Final TACs',
 'Matching - EU share',
 'Matching - TAC split share',
 'Matching - ICES area share']

In [45]:
pd.read_excel(os.path.realpath("data/icesTACcomparison.xlsx"), 1)

,Tab,Description
0,ICES advice,ICES advice by TAC and year
1,Council agreed TAC,Council agreed TAC by Member State and year
2,Comparison,Comparing ICES advice and agreed TACs by Membe...
3,Overall results,Calculates the difference between TACs and ICE...
4,Results by Member State,Calculates the difference between TACs and ICE...
5,Results by % difference,Calculates the difference between TACs and ICE...
6,Results by # of TACs,Calculates the number of TACs that exceed ICES...
7,Results by # of TACs by MS,Calculates the number of TACs that exceed ICES...
8,Results by third country share,Calculates the difference between TACs and ICE...
9,Results by species,Calculates the difference between TACs and ICE...


#### TAC

In [50]:
tac = pd.read_excel(os.path.realpath("data/icesTACcomparison.xlsx"), 'Council agreed TAC')

In [51]:
# extract acronym of species and convert to lower case
tac["speciesAcronym"] = tac["Reference"].str.extract(r"\(([\w\-]+)" , expand=False).str.lower()

In [229]:
tac[(tac.Species == 'Plaice') & (tac.Year.isin([2012,2016,2020])) & (tac.Level == 'TAC') & (sad_tac['ICES area'] == 4)  ]

,Reference,TAC ID,Species,TAC Zone,Level,TAC for comparison,Year,Amendment/Original,Sea,Amendment check,Pre-uplift/post-deduction,Uplift/deduction %,Post-uplift/pre-deduction,Notes,speciesAcronym
16399,(PLE/03AN.) - 2016 - TAC - Original,(PLE/03AN.),Plaice,Skagerrak,TAC,10056.410256,2016,Original,Atlantic,Final,10056.410256,0.170,11766,17% quota uplift,ple
16431,(PLE/03AN.) - 2020 - TAC - Original,(PLE/03AN.),Plaice,Skagerrak,TAC,19647,2020,Original,Atlantic,Final,19647,NaN,19647,NaN,ple
16808,(PLE/2A3AX4) - 2012 - TAC - Original,(PLE/2A3AX4),Plaice,4; Union waters of 2a; that part of 3a not cov...,TAC,84410,2012,Original,Atlantic,Final,84410,NaN,84410,NaN,ple
16862,(PLE/2A3AX4) - 2016 - TAC - Original,(PLE/2A3AX4),Plaice,4; Union waters of 2a; that part of 3a not cov...,TAC,128376.218324,2016,Original,Atlantic,Final,128376.218324,0.026,131714,2.6% quota uplift,ple
16898,(PLE/2A3AX4) - 2020 - TAC - Original,(PLE/2A3AX4),Plaice,4; Union waters of 2a; that part of 3a not cov...,TAC,146852,2020,Original,Atlantic,Final,146852,NaN,146852,NaN,ple


#### ICES advice

In [ ]:
sad = pd.read_excel(os.path.realpath("data/icesTACcomparison.xlsx"), 'ICES advice') 

In [ ]:
sad[(sad['ICES code'] == 'ple.27.420') & (sad.Year.isin([2012,2016,2020]))]

,Code,Species,ICES code,ICES area,Advice,Year,Sea,Choices,ICES advice,Catches corresponding to advice,Landings corresponding to advice,Unnamed: 11
1507,ple.27.420 - 2012,Plaice,ple.27.420,4,84410,2012,Atlantic,NaN,Apply first stage of the management plan,NaN,84410,NaN
1511,ple.27.420 - 2016,Plaice,ple.27.420,4,152910,2016,Atlantic,North sea split out,MSY approach,216345,152910,NaN
1515,ple.27.420 - 2020,Plaice,ple.27.420,4,166499,2020,Atlantic,North sea split out,MSY approach,166499,NaN,NaN


#### TAC-SAC Comparison

In [48]:
sad_tac = pd.read_excel(os.path.realpath("data/icesTACcomparison.xlsx"), 'Comparison') 

In [230]:
sad_tac[(sad_tac.Species == 'Plaice') & (sad_tac.Year.isin([2012,2016,2020])) & (sad_tac.Level == 'TAC') & (sad_tac['ICES area'] == 4)  ]

,Final Ref,Reference,TAC ID,Species,TAC Zone,ICES area,Level,TAC,Year,Amendment/Original,Sea,Amendment check,Agreement,ICES Advice,TAC above advice (t),TAC above advice (%),Net difference (t),Net difference (%),Advice change ICES,Advice change %,Advice change quantity,Previous year TAC,ICES with Council
16399,(PLE/03AN.) - 2016 - TAC - Final,(PLE/03AN.) - 2016 - TAC - Original,(PLE/03AN.),Plaice,Skagerrak,4,TAC,10056.410256,2016,Original,Atlantic,Final,Norway joint management,11108.11597,0.0,0.0,-1051.705714,-0.094679,0.766839,4.512713,-9093.11597,2015.0,11108.11597
16431,(PLE/03AN.) - 2020 - TAC - Final,(PLE/03AN.) - 2020 - TAC - Original,(PLE/03AN.),Plaice,Skagerrak,4,TAC,19647,2020,Original,Atlantic,Final,Norway joint management,19647,0.0,0.0,0.000000,0,0.170719,8.731055,-17628.00000,2019.0,19647
16808,(PLE/2A3AX4) - 2012 - TAC - Final,(PLE/2A3AX4) - 2012 - TAC - Original,(PLE/2A3AX4),Plaice,4; Union waters of 2a; that part of 3a not cov...,4,TAC,84410,2012,Original,Atlantic,Final,Norway joint management,84410,0.0,0.0,0.000000,0,0.314798,40.974142,-82399.00000,2011.0,84410
16862,(PLE/2A3AX4) - 2016 - TAC - Final,(PLE/2A3AX4) - 2016 - TAC - Original,(PLE/2A3AX4),Plaice,4; Union waters of 2a; that part of 3a not cov...,4,TAC,128376.218324,2016,Original,Atlantic,Final,Norway joint management,141801.88403,0.0,0.0,-13425.665706,-0.094679,0.104583,69.373143,-139786.88403,2015.0,141801.88403
16898,(PLE/2A3AX4) - 2020 - TAC - Final,(PLE/2A3AX4) - 2020 - TAC - Original,(PLE/2A3AX4),Plaice,4; Union waters of 2a; that part of 3a not cov...,4,TAC,146852,2020,Original,Atlantic,Final,Norway joint management,146852,0.0,0.0,0.000000,0,0.170742,71.735017,-144833.00000,2019.0,146852


### StockAssessment

In [395]:
stock = pd.read_csv(os.path.relpath('data/stockAssesment/StockAssessment.csv'), names=range(138))
stock.columns = stock.iloc[0,:]
stock = stock[1:]

C:\Users\isaldiviagonzatti\AppData\Local\Temp\ipykernel_5440\1294945985.py:1: DtypeWarning: Columns (0,1,3,4,6,10,12,13,14,17,18,19,20,21,22,23,27,28,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,46,47,48,49,50,51,52,54,56,57,58,59,60,62,65,66,68,69,70,71,72,73,74,75,76,79,82,85,88,91,94,97,100,103,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,137) have mixed types. Specify dtype option on import or set low_memory=False.
  stock = pd.read_csv(os.path.relpath('data/stockAssesment/StockAssessment.csv'), names=range(138))


In [396]:
# extract EN name and acronym of species
stock['enName'] = stock["StockDescription"].str.extract(r"^(.+?) ?(?:\d|\(|$)" , expand=False)
stock['speciesAcronym'] = stock["FishStock"].str.extract(r"^([^.]*).*" , expand=False)

In [397]:
# print(stock.columns.tolist())

In [411]:
# select useful years and columns
stock = stock.loc[:,['Year','enName', 'speciesAcronym','FishStock','StockKey', 'SpeciesName', "ICES Areas (splited with character '~')", 
                     'StockSize', 'StockSizeDescription', 'StockSizeUnits',
                      'FishingPressure', 'FishingPressureDescription', 'FishingPressureUnits',
                     'Flim', 'Fpa', 'Blim', 'Bpa', 'FMSY', 'MSYBtrigger', 
                     'CatchesLadingsUnits', 'Landings', 'OfficialLandings', 'Catches',
                     'Report']]
stock.Year = stock.Year.astype(int)
stock = stock[stock.Year.isin([2012,2016,2020])]
stock.columns

Index(['Year', 'enName', 'speciesAcronym', 'FishStock', 'StockKey',
       'SpeciesName', 'ICES Areas (splited with character '~')', 'StockSize',
       'StockSizeDescription', 'StockSizeUnits', 'FishingPressure',
       'FishingPressureDescription', 'FishingPressureUnits', 'Flim', 'Fpa',
       'Blim', 'Bpa', 'FMSY', 'MSYBtrigger', 'CatchesLadingsUnits', 'Landings',
       'OfficialLandings', 'Catches', 'Report'],
      dtype='object')

In [430]:
# To merge with OfficialCatches database
# Split the areas column into several columns and melt to get one row per species-area (as per in the CatchesOfficial) 


sepAreas = stock["ICES Areas (splited with character '~')"].str.split('~', expand=True)
sepAreas = sepAreas.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

stockMelt = stock.join(sepAreas)
stockMelt = stockMelt.melt(id_vars=['Year', 'enName', 'speciesAcronym', 'FishStock', 'StockKey',
       'SpeciesName', "ICES Areas (splited with character '~')", 'StockSize',
       'StockSizeDescription', 'StockSizeUnits', 'FishingPressure',
       'FishingPressureDescription', 'FishingPressureUnits', 'Flim', 'Fpa',
       'Blim', 'Bpa', 'FMSY', 'MSYBtrigger', 'CatchesLadingsUnits', 'Landings',
       'OfficialLandings', 'Catches', 'Report'], 
       var_name='fullArea', value_name='area')
stockMelt = stockMelt.dropna(subset=['area'])


# (later realised could have done it with explode)

stockExplode = (stock.set_index(['Year', 'enName', 'speciesAcronym', 'FishStock', 'StockKey',
       'SpeciesName', 'StockSize',
       'StockSizeDescription', 'StockSizeUnits', 'FishingPressure',
       'FishingPressureDescription', 'FishingPressureUnits', 'Flim', 'Fpa',
       'Blim', 'Bpa', 'FMSY', 'MSYBtrigger', 'CatchesLadingsUnits', 'Landings',
       'OfficialLandings', 'Catches', 'Report'])
   .apply(lambda x: x.str.split('~').explode())
   .reset_index())  
stockExplode["ICES Areas (splited with character '~')"] = stockExplode["ICES Areas (splited with character '~')"].str.strip()

In [445]:
stockMelt

,Year,enName,speciesAcronym,FishStock,StockKey,SpeciesName,ICES Areas (splited with character '~'),StockSize,StockSizeDescription,StockSizeUnits,FishingPressure,FishingPressureDescription,FishingPressureUnits,Flim,Fpa,Blim,Bpa,FMSY,MSYBtrigger,CatchesLadingsUnits,Landings,OfficialLandings,Catches,Report,fullArea,area
0,2012,Sandeel,san,san.sa.1r,169246,Ammodytes,27.4.b ~ 27.4.c,152970,SSB,tonnes,0.112,F,Year-1,NaN,NaN,110000,145000,NaN,NaN,tonnes,NaN,NaN,45954,https://doi.org/10.17895/ices.advice.10000,0,27.4.b
1,2016,Sandeel,san,san.sa.1r,169246,Ammodytes,27.4.b ~ 27.4.c,231422,SSB,tonnes,0.028,F,Year-1,NaN,NaN,110000,145000,NaN,NaN,tonnes,NaN,NaN,15407,https://doi.org/10.17895/ices.advice.10000,0,27.4.b
2,2020,Sandeel,san,san.sa.1r,169246,Ammodytes,27.4.b ~ 27.4.c,60901,SSB,tonnes,0.515,F,Year-1,NaN,NaN,110000,145000,NaN,NaN,tonnes,NaN,NaN,108944,https://doi.org/10.17895/ices.advice.10000,0,27.4.b
3,2012,Sandeel,san,san.sa.2r,169247,Ammodytes,27.4.b ~ 27.4.c,42319,SSB,tonnes,0.153,F,ratio,NaN,NaN,56000,84000,NaN,NaN,tonnes,NaN,NaN,12672,https://doi.org/10.17895/ices.advice.10001,0,27.4.b
4,2016,Sandeel,san,san.sa.2r,169247,Ammodytes,27.4.b ~ 27.4.c,26796,SSB,tonnes,0.188,F,ratio,NaN,NaN,56000,84000,NaN,NaN,tonnes,NaN,NaN,9569,https://doi.org/10.17895/ices.advice.10001,0,27.4.b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35447,2016,Porbeagle,por,por.27.nea,169193,Lamna nasus,27.1.a ~ 27.1.b ~ 27.10.a.1 ~ 27.10.a.2 ~ 27.1...,0.296623,B/Bmsy,NaN,0.033227,F/Fmsy,NaN,NaN,NaN,0.333,NaN,1.0,0.5,NaN,11.0,NaN,NaN,https://doi.org/10.17895/ices.advice.19754584.v1,64,27.9.b.2
35448,2020,Porbeagle,por,por.27.nea,169193,Lamna nasus,27.1.a ~ 27.1.b ~ 27.10.a.1 ~ 27.10.a.2 ~ 27.1...,0.404563,B/Bmsy,NaN,0.016643,F/Fmsy,NaN,NaN,NaN,0.333,NaN,1.0,0.5,NaN,7.0,NaN,NaN,https://doi.org/10.17895/ices.advice.19754584.v1,64,27.9.b.2
35503,2012,Spurdog,dgs,dgs.27.nea,169094,Squalus acanthias,27.1.a ~ 27.1.b ~ 27.10.a.1 ~ 27.10.a.2 ~ 27.1...,45009.6,SSB,tonnes,0.024032,Frel,NaN,NaN,NaN,240569.0,336796.0,NaN,336796.0,tonnes,447.49185,NaN,5203.995404,https://doi.org/10.17895/ices.advice.19753588.v1,64,27.9.b.2
35504,2016,Spurdog,dgs,dgs.27.nea,169094,Squalus acanthias,27.1.a ~ 27.1.b ~ 27.10.a.1 ~ 27.10.a.2 ~ 27.1...,53772.9,SSB,tonnes,0.00681,Frel,NaN,NaN,NaN,240569.0,336796.0,NaN,336796.0,tonnes,371.2235,NaN,1951.299516,https://doi.org/10.17895/ices.advice.19753588.v1,64,27.9.b.2


In [444]:
stockMelt[(stockMelt.FishStock== 'san.sa.1r') & (stockMelt["ICES Areas (splited with character '~')"] == '27.4.b ~ 27.4.c') & (stockMelt.Year.isin([2016])) ]

,Year,enName,speciesAcronym,FishStock,StockKey,SpeciesName,ICES Areas (splited with character '~'),StockSize,StockSizeDescription,StockSizeUnits,FishingPressure,FishingPressureDescription,FishingPressureUnits,Flim,Fpa,Blim,Bpa,FMSY,MSYBtrigger,CatchesLadingsUnits,Landings,OfficialLandings,Catches,Report,fullArea,area
1,2016,Sandeel,san,san.sa.1r,169246,Ammodytes,27.4.b ~ 27.4.c,231422,SSB,tonnes,0.028,F,Year-1,NaN,NaN,110000,145000,NaN,NaN,tonnes,NaN,NaN,15407,https://doi.org/10.17895/ices.advice.10000,0,27.4.b
550,2016,Sandeel,san,san.sa.1r,169246,Ammodytes,27.4.b ~ 27.4.c,231422,SSB,tonnes,0.028,F,Year-1,NaN,NaN,110000,145000,NaN,NaN,tonnes,NaN,NaN,15407,https://doi.org/10.17895/ices.advice.10000,1,27.4.c


In [401]:
stock[(stock.speciesAcronym== 'san') & (stock["ICES Areas (splited with character '~')"] == '27.4.b ~ 27.4.c') & (stock.Year.isin([2016])) ]

,Year,enName,speciesAcronym,FishStock,StockKey,SpeciesName,ICES Areas (splited with character '~'),StockSize,StockSizeDescription,StockSizeUnits,FishingPressure,FishingPressureDescription,FishingPressureUnits,Flim,Fpa,Blim,Bpa,FMSY,MSYBtrigger,CatchesLadingsUnits,Landings,OfficialLandings,Catches,Report,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64
34,2016,Sandeel,san,san.sa.1r,169246,Ammodytes,27.4.b ~ 27.4.c,231422,SSB,tonnes,0.028,F,Year-1,NaN,NaN,110000,145000,NaN,NaN,tonnes,NaN,NaN,15407,https://doi.org/10.17895/ices.advice.10000,27.4.b,27.4.c,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
74,2016,Sandeel,san,san.sa.2r,169247,Ammodytes,27.4.b ~ 27.4.c,26796,SSB,tonnes,0.188,F,ratio,NaN,NaN,56000,84000,NaN,NaN,tonnes,NaN,NaN,9569,https://doi.org/10.17895/ices.advice.10001,27.4.b,27.4.c,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [96]:
sepAreas = stock["ICES Areas (splited with character '~')"].str.split('~', expand=True)
sepAreas = sepAreas.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [60]:
list1 = stock["ICES Areas (splited with character '~')"].to_list()

newList= []
for item in list1:
    if(str(item) != "nan"):
        newList.append(item)

max_len = -1
for ele in newList: 
    if(len(ele) > max_len): 
        max_len = len(ele) 
        res = ele 
        
print("Longest String is : ", res) 

Longest String is :  27.1.a ~ 27.1.b ~ 27.10.a.1 ~ 27.10.a.2 ~ 27.10.b ~ 27.12.a.1 ~ 27.12.a.2 ~ 27.12.a.3 ~ 27.12.a.4 ~ 27.12.b ~ 27.12.c ~ 27.14.a ~ 27.14.b.1 ~ 27.14.b.2 ~ 27.2.a.1 ~ 27.2.a.2 ~ 27.2.b.1 ~ 27.2.b.2 ~ 27.3.a ~ 27.3.b.23 ~ 27.3.c.22 ~ 27.3.d.24 ~ 27.3.d.25 ~ 27.3.d.26 ~ 27.3.d.27 ~ 27.3.d.28.1 ~ 27.3.d.28.2 ~ 27.3.d.29 ~ 27.3.d.30 ~ 27.3.d.31 ~ 27.3.d.32 ~ 27.4.a ~ 27.4.b ~ 27.4.c ~ 27.5.a.1 ~ 27.5.a.2 ~ 27.5.b.1.a ~ 27.5.b.1.b ~ 27.5.b.2 ~ 27.6.a ~ 27.6.b.1 ~ 27.6.b.2 ~ 27.7.a ~ 27.7.b ~ 27.7.c.1 ~ 27.7.c.2 ~ 27.7.d ~ 27.7.e ~ 27.7.f ~ 27.7.g ~ 27.7.h ~ 27.7.j.1 ~ 27.7.j.2 ~ 27.7.k.1 ~ 27.7.k.2 ~ 27.8.a ~ 27.8.b ~ 27.8.c ~ 27.8.d.1 ~ 27.8.d.2 ~ 27.8.e.1 ~ 27.8.e.2 ~ 27.9.a ~ 27.9.b.1 ~ 27.9.b.2


In [377]:
pleStock = stock[stock.FishStock == 'san.sa.1r']
pleStock = pleStock["ICES Areas (splited with character '~')"].str.split('~', expand=True).iloc[0,:].to_list()
pleStock = [s.strip() for s in pleStock]

['27.7.b',
 '27.7.c.1',
 '27.7.c.2',
 '27.7.e',
 '27.7.f',
 '27.7.g',
 '27.7.h',
 '27.7.j.1',
 '27.7.j.2',
 '27.7.k.1',
 '27.7.k.2']

In [64]:
# check what what data is in the custom columns
# stock[~stock.CustomUnits6.isna()].dropna(axis=1)

### OfficialNominalCatches

In [500]:
catches = pd.read_csv(os.path.realpath('data\OfficialNominalCatches\ICESCatchDataset2006-2020.csv')) 

In [501]:
# add country name column
catches['geo'] = catches.Country.map(abbrev_to_country).fillna(catches.Country)

# filter countries of interest
catches = catches[catches.geo.isin(countries)]

# convert Species to lower case
catches.Species = catches.Species.str.lower()

# 2020 has word characters, we divide it into two 
catches[['2020','2020c']] = catches['2020'].str.split(" +",expand = True) 
catches['2020'] = catches['2020'].astype(np.float64)

# keep useful columns
catches = catches.loc[:,['Country','geo','Species','Area','2012','2016','2020']]

In [502]:
# stockMelt.merge(catches, left_on=['speciesAcronym','area'], right_on=['Species', 'Area'])
catches = catches.melt(id_vars=['Country', 'geo','Species', 'Area'], 
       var_name='Year', value_name='CatchesCountry')

In [519]:
catchesPivot = catches.pivot_table( columns='geo', index=['Year','Area', 'Species'] , values='CatchesCountry', aggfunc='sum').reset_index()
catchesPivot['Year'] = catchesPivot['Year'].astype(int)

In [535]:
catchesPivot[(catchesPivot['Species'] == 'had') ].Area.unique()

array(['27', '27.1', '27.1.a', '27.1.b', '27.12', '27.12.a', '27.12.a_NK',
       '27.12.c', '27.12_NK', '27.14', '27.14.a', '27.14.b', '27.14.b.2',
       '27.14.b_NK', '27.14_NK', '27.1_NK', '27.2', '27.2.a', '27.2.a.1',
       '27.2.a.2', '27.2.a_NK', '27.2.b', '27.2.b.1', '27.2.b.2',
       '27.2.b_NK', '27.2_NK', '27.3', '27.3.a', '27.3.a.20', '27.3.a.21',
       '27.3.a_NK', '27.3.b.23', '27.3.c.22', '27.3.d', '27.3.d.24',
       '27.3.d.25', '27.3.d.26', '27.3.d.28', '27.3.d.28.1',
       '27.3.d.28.2', '27.3.d_NK', '27.4', '27.4.a', '27.4.b', '27.4.c',
       '27.4_NK', '27.5', '27.5.a', '27.5.a_NK', '27.5.b', '27.5.b.1',
       '27.5.b.1.b', '27.5.b.1_NK', '27.5.b.2', '27.5.b_NK', '27.5_NK',
       '27.6', '27.6.a', '27.6.b', '27.6.b.1', '27.6.b.2', '27.6.b_NK',
       '27.6_NK', '27.7', '27.7.a', '27.7.b', '27.7.c', '27.7.c.2',
       '27.7.c_NK', '27.7.d', '27.7.e', '27.7.f', '27.7.g', '27.7.h',
       '27.7.j', '27.7.j.1', '27.7.j.2', '27.7.j_NK', '27.7.k',
       '27.7.k.1

In [532]:
mStockCatch = stockMelt.merge(catchesPivot, left_on=['speciesAcronym','area','Year'], right_on=['Species', 'Area', 'Year'], how='left')
mStockCatch[mStockCatch.Area.isna()]

,Year,enName,speciesAcronym,FishStock,StockKey,SpeciesName,ICES Areas (splited with character '~'),StockSize,StockSizeDescription,StockSizeUnits,FishingPressure,FishingPressureDescription,FishingPressureUnits,Flim,Fpa,Blim,Bpa,FMSY,MSYBtrigger,CatchesLadingsUnits,Landings,OfficialLandings,Catches,Report,fullArea,area,Area,Species,Belgium,Denmark,Estonia,Finland,France,Germany,Ireland,Latvia,Lithuania,Netherlands,Poland,Portugal,Spain,Sweden,United Kingdom of GB
15,2012,Haddock,had,had.27.5a,169112,Melanogrammus aeglefinus,27.5.a.1 ~ 27.5.a.2,76375,SSB/B45cm,tonnes,0.3964,F/Harvest rate,NaN,NaN,NaN,35500,49400,NaN,NaN,NaN,46032,NaN,46032,https://doi.org/10.17895/ices.advice.19447949.v1,0,27.5.a.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2016,Haddock,had,had.27.5a,169112,Melanogrammus aeglefinus,27.5.a.1 ~ 27.5.a.2,72500,SSB/B45cm,tonnes,0.3366,F/Harvest rate,NaN,NaN,NaN,35500,49400,NaN,NaN,NaN,38090,NaN,38090,https://doi.org/10.17895/ices.advice.19447949.v1,0,27.5.a.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,2020,Haddock,had,had.27.5a,169112,Melanogrammus aeglefinus,27.5.a.1 ~ 27.5.a.2,86713,SSB/B45cm,tonnes,0.416,F/Harvest rate,NaN,NaN,NaN,35500,49400,NaN,NaN,NaN,54459,NaN,54459,https://doi.org/10.17895/ices.advice.19447949.v1,0,27.5.a.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,2012,Flounder,bzq,bzq.27.2628,233880,Platichthys,27.3.d.26 ~ 27.3.d.28,134.202662854112,SSB,Kilograms per hour,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tonnes,3958.88978,NaN,NaN,https://doi.org/10.17895/ices.advice.19928909.v1,0,27.3.d.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,2016,Flounder,bzq,bzq.27.2628,233880,Platichthys,27.3.d.26 ~ 27.3.d.28,51.6361314042807,SSB,Kilograms per hour,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tonnes,4251.5075,NaN,4442.455,https://doi.org/10.17895/ices.advice.19928909.v1,0,27.3.d.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3613,2016,Spurdog,dgs,dgs.27.nea,169094,Squalus acanthias,27.1.a ~ 27.1.b ~ 27.10.a.1 ~ 27.10.a.2 ~ 27.1...,53772.9,SSB,tonnes,0.00681,Frel,NaN,NaN,NaN,240569.0,336796.0,NaN,336796.0,tonnes,371.2235,NaN,1951.299516,https://doi.org/10.17895/ices.advice.19753588.v1,63,27.9.b.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3614,2020,Spurdog,dgs,dgs.27.nea,169094,Squalus acanthias,27.1.a ~ 27.1.b ~ 27.10.a.1 ~ 27.10.a.2 ~ 27.1...,63784.9,SSB,tonnes,0.00416,Frel,NaN,NaN,NaN,240569.0,336796.0,NaN,336796.0,tonnes,525.67275,NaN,1468.03534,https://doi.org/10.17895/ices.advice.19753588.v1,63,27.9.b.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3624,2012,Spurdog,dgs,dgs.27.nea,169094,Squalus acanthias,27.1.a ~ 27.1.b ~ 27.10.a.1 ~ 27.10.a.2 ~ 27.1...,45009.6,SSB,tonnes,0.024032,Frel,NaN,NaN,NaN,240569.0,336796.0,NaN,336796.0,tonnes,447.49185,NaN,5203.995404,https://doi.org/10.17895/ices.advice.19753588.v1,64,27.9.b.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3625,2016,Spurdog,dgs,dgs.27.nea,169094,Squalus acanthias,27.1.a ~ 27.1.b ~ 27.10.a.1 ~ 27.10.a.2 ~ 27.1...,53772.9,SSB,tonnes,0.00681,Frel,NaN,NaN,NaN,240569.0,336796.0,NaN,336796.0,tonnes,371.2235,NaN,1951.299516,https://doi.org/10.17895/ices.advice.19753588.v1,64,27.9.b.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [492]:
denmarkPle = catches[(catches['Area'].isin(pleStock))  & (catches['Species'] == 'san')].copy()
denmarkPle['2020'] = denmarkPle['2020'].astype(np.float64)
denmarkPle

,Country,geo,Species,Area,2012,2016,2020
40092,DK,Denmark,san,27.7.e,0.00,0.00,0.0000
40121,FR,France,san,27.7.b,0.08,0.00,0.0000
40123,FR,France,san,27.7.e,35.66,34.27,22.3620
40124,FR,France,san,27.7.h,0.07,0.03,0.4890
40137,GB,United Kingdom of GB,san,27.7.b,0.00,0.00,0.0000
40159,LT,Lithuania,san,27.7.b,0.00,0.00,0.0000
53930,GB,United Kingdom of GB,san,27.7.e,5.30,0.33,0.0020
53931,GB,United Kingdom of GB,san,27.7.f,14.00,20.58,5.4323


### TAC/SAD